# Regress Mortgage Approvals on Race, Sex, and Other Variables

The purpose of this program is to regress a mortgage approval variable against race, ethnicity, gender, and other control variables found in HMDA data. This script will be designed for a distributive environment using the model below.

$P(Approval = 1 | Race/Sex, \chi_ji, \alpha_i) = \beta_0 + \lambda_ji * Race/Sex + \beta_ji * \chi_ji + \alpha_i + \mu $

Where $\lambda_ji$ are the variables of interest, $\beta_ji$ are the coefficients on the control variables, 
alpha_i are the fixed effects, and $\chi_j$ are the control variables.

Variables of Interest
- White
- Black
- Asian
- Hispanic
- Other
- Male 
- Female

Control Variables
- Income (log)
- Loan to Value ratio
- Debt to Income ratio
- Loan Amount (log)
- Pre-Approval indicators

Variables ommited in model to prevent perfect collinearity.
- Race - White
- Sex - Male

Filters
- Loan Purpose
- Occupancy Type

Clustered Standard errors
- by Lender
- by State
- by County
- by Census Tract

In [55]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import sys
print("Python Version", sys.version)
tf.test.is_built_with_cuda()
print("TF Version", tf.version.VERSION)

ModuleNotFoundError: No module named 'matplotlib'

- FE: https://www.youtube.com/watch?v=FCm3_Id6RKM
- Custom Model: https://www.youtube.com/watch?v=S6tLSI8bjGs

### Load in Dataset

In [56]:
# Load in HMDA Data
HMDA = r'HMDA Clean IL SAMPLE.csv'
HMDA = pd.read_csv(HMDA)
#HMDA

### Further Cleaning

In [57]:
#Clean df
HMDA_clean_1 = HMDA.copy()
HMDA_clean_1 = HMDA_clean_1.dropna()
HMDA_clean_1['Census_Tract'] = HMDA_clean_1['Census_Tract'].apply(str)

#Filter Occupancy type to Principoal residence. Omits secondary residence purposes and investment purposes.
# "Occupancy_Type" = 1, Second Residence" = 2, "Investment Property" = 3.
HMDA_clean_2_1 = HMDA_clean_1[HMDA_clean_1["Occupancy_Type"] == 1]

#Sets County_Code and Census_Tract as strings.
HMDA_clean_2 = HMDA_clean_2_1.copy()
HMDA_clean_2['County_Code'] = HMDA_clean_2['County_Code'].astype(str)
HMDA_clean_2['Census_Tract'] = HMDA_clean_2['Census_Tract'].astype(str)
HMDA_clean_2

#Sets dummy variables for different columns
HMDA_clean_3 = pd.get_dummies(HMDA_clean_2, columns = ['Year','Race', 'Sex', 'DTI_Ratio', 'Loan_Type'])
HMDA_clean_3.columns

Index(['index', 'Lender_LEI', 'State', 'County_Code', 'Census_Tract',
       'Approved', 'Denied', 'Income', 'Log_Income', 'Loan_Amount',
       'Log_Loan_Amount', 'LTV', 'Preapproval', 'Occupancy_Type', 'Year_2019',
       'Year_2020', 'Year_2021', 'Race_0_White', 'Race_Asian', 'Race_Black',
       'Race_Latinx', 'Race_Other', 'Sex_0_Male', 'Sex_Female',
       'DTI_Ratio_0%-20%', 'DTI_Ratio_20%-<30%', 'DTI_Ratio_30%-<36%',
       'DTI_Ratio_36', 'DTI_Ratio_37', 'DTI_Ratio_38', 'DTI_Ratio_39',
       'DTI_Ratio_40', 'DTI_Ratio_41', 'DTI_Ratio_42', 'DTI_Ratio_43',
       'DTI_Ratio_44', 'DTI_Ratio_45', 'DTI_Ratio_46', 'DTI_Ratio_47',
       'DTI_Ratio_48', 'DTI_Ratio_49', 'DTI_Ratio_50%-60%', 'DTI_Ratio_>60%',
       'Loan_Type_Conventional', 'Loan_Type_FHA', 'Loan_Type_RHS or FSA',
       'Loan_Type_VA'],
      dtype='object')

In [58]:
#Remove Columns to prevent Multicollinearity, remove index, and remove outcome variables. 
HMDA_clean_4 = HMDA_clean_3.drop(['Race_0_White', 'Sex_0_Male', 'DTI_Ratio_0%-20%','index'], axis = 1)
df_clean = HMDA_clean_4.copy()

### Create Fixed Effects Set

### Model 1 Training Set

In [59]:
#Remove y variable and columns not to be used
train_set_1 = df_clean.drop(['Denied', 'Occupancy_Type', 'Lender_LEI', 'County_Code',
                            'Census_Tract', 'Income', 'Loan_Amount', 'Preapproval','State'], axis = 1)
train_1 = np.array(train_set_1)
x_1 = train_set_1.drop(['Approved'], axis = 1)
y_1 = train_set_1['Approved']

train_set_1.shape

(184853, 34)

# Model 1

In [60]:
[len(x_1.keys())]

[33]

In [86]:
def build_sgd_model(training_set):
    model = keras.Sequential([ 
        layers.Dense(1, input_shape = [len(x_1.keys())], activation = 'relu', name = 'input')
        #,layers.Dense(1, name = 'output')
    ])
        
    
    optimizer = tf.keras.optimizers.SGD(learning_rate = .01, momentum = .3)
    
    model.compile(loss = 'mse',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse', 'accuracy'])
    return model
    
model_1 = build_sgd_model(train_set_1)

In [87]:
model_1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 1)                 34        
Total params: 34
Trainable params: 34
Non-trainable params: 0
_________________________________________________________________


### Train The Model

In [88]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end = '')

Epochs = 100

history = model_1.fit(
    x_1,
    y_1,
    epochs = Epochs,
    verbose = 0,
    callbacks = [PrintDot()],
    validation_split = 0,
    use_multiprocessing = True
    )


....................................................................................................

In [89]:
tf.keras.utils.plot_model(model_1, to_file = 'simple.png', show_shapes = True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [90]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,loss,mae,mse,accuracy,epoch
0,1.062809,0.921930,1.062809,0.083061,0
1,0.917113,0.917113,0.917113,0.082887,1
2,0.917113,0.917113,0.917113,0.082887,2
3,0.917113,0.917113,0.917113,0.082887,3
4,0.917113,0.917113,0.917113,0.082887,4
...,...,...,...,...,...
95,0.917113,0.917113,0.917113,0.082887,95
96,0.917113,0.917113,0.917113,0.082887,96
97,0.917113,0.917113,0.917113,0.082887,97
98,0.917113,0.917113,0.917113,0.082887,98


In [91]:
model_1.save('test model.h5')

In [92]:
model_1.evaluate(x_1, y_1, callbacks = [PrintDot()],use_multiprocessing = True)

5777/5777 [==============================] - 7s 1ms/step - loss: 0.9171 - mae: 0.9171 - mse: 0.9171 - accuracy: 0.0829


[0.9171125292778015,
 0.9171125292778015,
 0.9171125292778015,
 0.08288748562335968]

In [93]:
cheese = model_1.get_layer(index = 0)
model_1.weights

[<tf.Variable 'input/kernel:0' shape=(33, 1) dtype=float32, numpy=
 array([[-3.86505508e+00],
        [-9.79351521e+00],
        [-7.27807999e+01],
        [-2.73064613e-01],
        [-7.57032216e-01],
        [ 4.61148359e-02],
        [-1.53226897e-01],
        [-1.80995315e-02],
        [ 7.50448704e-02],
        [-3.14336687e-01],
        [-7.63748512e-02],
        [-5.30496478e-01],
        [-3.38496298e-01],
        [-1.12133786e-01],
        [-1.75352603e-01],
        [ 3.20575118e-01],
        [-9.50981826e-02],
        [-3.61385196e-01],
        [ 4.04084116e-01],
        [ 3.90894502e-01],
        [-2.63809174e-01],
        [ 9.39038992e-02],
        [-1.77662537e-01],
        [-3.66168022e-01],
        [-1.02943115e-01],
        [-8.31515491e-02],
        [ 3.67494076e-02],
        [-4.70138788e-01],
        [-3.50581586e-01],
        [-7.24182785e-01],
        [-3.38348776e-01],
        [-1.54225767e-01],
        [ 3.73843551e-01]], dtype=float32)>,
 <tf.Variable 'input/bia

In [19]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mse']
    

SyntaxError: unexpected EOF while parsing (466573885.py, line 9)

In [47]:
class HMDARegress(keras.Sequential):
    
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(1, activation = 'relu', name = 'output')
        
    def call(self, inpus, training = False):
        x = self.dense(inputs)
        return x
        
        
model = HMDARegress()
model.call

<bound method HMDARegress.call of <__main__.HMDARegress object at 0x000002B1422C1580>>

In [ ]:
def linreg(x, y, Epochs):
    model = tf.keras.experimental.LinearModel()        
    
    optimizer = tf.keras.optimizers.SGD(learning_rate = .01, momentum = .3)
    
    model.compile(loss = 'mse',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse'])
    
    model.fit(x, y, epochs = Epochs)
    
    return model
    
model_1 = linreg(x_1, y_1, 5)

Epoch 1/5
5777/5777 [==============================] - 8s 1ms/step - loss: nan - mae: nan - mse: nan
Epoch 2/5
5777/5777 [==============================] - 7s 1ms/step - loss: nan - mae: nan - mse: nan
Epoch 3/5
5777/5777 [==============================] - 7s 1ms/step - loss: nan - mae: nan - mse: nan
Epoch 4/5
5777/5777 [==============================] - 8s 1ms/step - loss: nan - mae: nan - mse: nan
Epoch 5/5
5777/5777 [==============================] - 8s 1ms/step - loss: nan - mae: nan - mse: nan


In [71]:
model_1.save('test1.tf')

INFO:tensorflow:Assets written to: test1.tf\assets


In [91]:
cheese = model_1.get_layer(index = 0)
cheese.get_weights()

[array([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], dtype=float32)]